City for a pleasant life

1. A description of the problem

I have a good friend.
And I want to help him.

My friend is a famous artist.
And now he is looking for a city to which he could go to work.
Of course, first of all he is interested in art.
As for all people of art, the environment, the atmosphere in which he will live, means a lot to him.

Suppose they have the opportunity to move to one of several cities. In one of the areas of these cities. Let it be Toronto and his districts (or Florence, Stockholm or Minsk ...).
I want, using data analysis, to determine which district of which city will be the most pleasant for my friend's family.

His family consists of three people - himsself, his wife and daughter. And he wants each of them to get what he likes in the new city.

His wife is a very good cook. He knows the cuisine of different countries. But he wants to know even more.
Their daughter loves all the tasty things - juices, ice cream, fruit.
My friend is a subtle connoisseur of wine and coffee. But does not like beer.

1.1. Audience of my project and why they would care about this problem

The audience of my project is people who want to change their place of residence.
Or just tourists who want to choose not just a place where you can live for several weeks.
For this, sites https://www.booking.com or https://www.airbnb.ru are enough.
And the place where it can be done with the greatest pleasure.
Why will they take care of this problem?
Because it is new for them, interesting and necessary.
And it costs almost nothing)

2. Discussion of the background

It is clear that the wife agrees to live in the district of city where there are many cafes and a variety of restaurants - Italian, Indian, Chinese, Japanese ...
Daughter - where there are many children's cafes, ice cream cafes, flower shops ...
My friend - where there are liquor stores, bars, but not pubs)
                     * * * * *

Obviously, in order to be able to make a decision, the task must be formalized.
Let the number of places in the area, pleasant for the wife - W. Number of places pleasant for a child - D. Number of places pleasant for a friend - F.
We will also assume that the result of the choice of each family member affects to varying degrees. The coefficient for choosing a friend is f, wives — w, daughters — d. And f + c + d = 1.
It is also important to consider how many people live in the area. If there are a lot of them, this is not the best option.

Thus, the task is reduced to the fact that you need to find a DS district in one of the cities S, such that
     R (S, DS) = (F * f + W * w + D * d) / P                     (*)

where P is the population of the district, will be the greatest.
You can even more detail:
     R (S, DS1, ..., DSn) = (F * f + W * w + D * d) / P          (**)

3. A description of the data

Wikipedia (city's districts, population): https://en.wikipedia.org
Google Maps Geocoding API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/
Geographical coordinates for postal codes: http://cocl.us/Geospatial_data
Foursquare API: https://api.foursquare.com

4. How it will be used to solve the problem

From the pages of Wikipedia to obtain data on the administrative-territorial division of cities.
About the population of cities and their areas.
Associate the information found with the geographical coordinates of the areas.
Get, using the service Foursquare, information about places located in these areas.
Select those places that interest family members, and count their number.
Apply the formula (*) or (**) to the found information.
Choose the best result and recommend it to my friend.
Listen to his comments (or suggestions) and try to search for information in a new way.

5. Methodology section

5.1. Import libraryes

In [11]:

import pandas as pd
import pickle

In [12]:

import folium
import numpy as np
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
df = pickle.load(open('df.pkl', 'rb'))
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'df.pkl'

In [15]:
data = {'Borough':[], 'Neighbourhood':[], 'Neighbourhood_Latitude':[],
        'Neighbourhood_Longitude':[], 'Venue':[], 'Venue_Latitude':[],
        'Venue_Longitude':[], 'Venue_Category':[]}
data = pd.DataFrame(data = data)
data

,Borough,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category


In [17]:
CLIENT_ID = 'V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS' # your Foursquare ID
CLIENT_SECRET = 'TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO' # your Foursquare Secret
VERSION = '20190119' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V1LUIIHLLAVCATP3WOM2A2B4XDI1HZ1IIJWOJ31UVU0JKLLS
CLIENT_SECRET:TU20VTP045ZAWBOXACDX2JFIJDINS2Q5O0GYB1GRCPVS5TDO


In [18]:
for iNeigh in df.index:

    neighbourhood_latitude = df.loc[iNeigh, 'Latitude'] # neighbourhood latitude value
    neighbourhood_longitude = df.loc[iNeigh, 'Longitude'] # neighbourhood longitude value
    neighbourhood_name = df.loc[iNeigh, 'Neighbourhood'] # neighbourhood name
    
    # type your answer here
    LIMIT = 1000 # limit of number of venues returned by Foursquare API

    radius = 1000 # define radius

    # create URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighbourhood_latitude, 
        neighbourhood_longitude, 
        radius, 
        LIMIT)
#    url # display URL
    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    if nearby_venues.shape == (0,0):
        continue
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
    
    for iv in nearby_venues.index:
        data = data.append(
            {'Borough': df.loc[iNeigh, 'Borough'], 
             'Neighbourhood': df.loc[iNeigh, 'Neighbourhood'],
             'Neighbourhood_Latitude': df.loc[iNeigh, 'Latitude'],
             'Neighbourhood_Longitude': df.loc[iNeigh, 'Longitude'],
             'Venue': nearby_venues.loc[iv, 'name'],
             'Venue_Latitude': nearby_venues.loc[iv, 'lat'],
             'Venue_Longitude': nearby_venues.loc[iv, 'lng'],
             'Venue_Category': nearby_venues.loc[iv, 'categories']}, ignore_index=True)

NameError: name 'df' is not defined

In [9]:
data.columns = ['Borough', 'Neighbourhood', 'Neighbourhood_Latitude',
        'Neighbourhood_Longitude', 'Venue', 'Venue_Category',
        'Venue_Latitude', 'Venue_Longitude']

In [10]:
data

,Borough,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Category,Venue_Latitude,Venue_Longitude


In [11]:
data.shape

(0, 8)

In [12]:
pickle.dump(data, open('0-102.pkl', 'wb'))

In [13]:
data = pickle.load(open('0-102.pkl', 'rb'))

In [14]:
data['Venue_Category'].unique()

array([], dtype=float64)

In [15]:
my_categories = {'F': [], 'W': [], 'D': []}

In [16]:

for i in data.index:
    data.loc[i,'Cat'] = 'F' if data.loc[i,'Venue_Category'] in my_categories['F'] else 'W' if data.loc[i,'Venue_Category'] in my_categories['W'] else 'D'

In [17]:
db = data[['Borough', 'Neighbourhood', 'Cat', 'Venue']].groupby(['Borough', 'Neighbourhood', 'Cat'], sort=(True, True, False)).count()
db

KeyError: "['Cat'] not in index"

In [18]:
df['D'] = 0
df['F'] = 0
df['W'] = 0

for ind in db.index:
    df_ind = df[(df.Borough == ind[0]) & (df.Neighbourhood == ind[1])].index[0]
    df.at[df_ind,ind[2]]  = int(db.loc[ind][0])
df

NameError: name 'df' is not defined

In [19]:
gdata = {'f >> w > d': [], 'f < w << d': [], 'w >> f > d': [], 'f = w = d': []}
gnsdata = {'f >> w > d': [], 'f < w << d': [], 'w >> f > d': [], 'f = w = d': []}
cdata = pd.DataFrame(columns=['Latitude', 'Longitude', 'Cluster', 'Borough', 'Neighbourhood'])

In [20]:
scheme, f, w, d = 'f >> w > d', 0.7, 0.2, 0.1

df['Like'] = f*df['F'] + w*df['W'] + d*df['D']
td = df.sort_values(by = 'Like', ascending=False)
gdata[scheme] = td['Like'].values.tolist()
gnsdata[scheme] = df['Like'].values.tolist()
for i in range(5):
    cdata = cdata.append({'Cluster':0, 
                          'Latitude': td.loc[i,'Latitude'], 'Longitude': td.loc[i,'Longitude'], 
                          'Borough': td.loc[i,'Borough'], 'Neighbourhood': td.loc[i,'Neighbourhood']},
           ignore_index=True)
td.head()

NameError: name 'df' is not defined

In [21]:
scheme, f, w, d = 'f < w << d', 0.1, 0.2, 0.7
df['Like'] = f*df['F'] + w*df['W'] + d*df['D']
gdata[scheme] = df.sort_values(by = 'Like', ascending=False)['Like'].values.tolist()
gnsdata[scheme] = df['Like'].values.tolist()
for i in range(5):
    cdata = cdata.append({'Cluster':1, 
                          'Latitude': td.loc[i,'Latitude'], 'Longitude': td.loc[i,'Longitude'], 
                          'Borough': td.loc[i,'Borough'], 'Neighbourhood': td.loc[i,'Neighbourhood']},
           ignore_index=True)
df.sort_values(by = 'Like', ascending=False).head()

NameError: name 'df' is not defined

In [23]:
scheme, f, w, d = 'w >> f > d', 0.2, 0.7, 0.1
df['Like'] = f*df['F'] + w*df['W'] + d*df['D']
gdata[scheme] = df.sort_values(by = 'Like', ascending=False)['Like'].values.tolist()
gnsdata[scheme] = df['Like'].values.tolist()
for i in range(5):
    cdata = cdata.append({'Cluster':2, 
                          'Latitude': td.loc[i,'Latitude'], 'Longitude': td.loc[i,'Longitude'], 
                          'Borough': td.loc[i,'Borough'], 'Neighbourhood': td.loc[i,'Neighbourhood']},
           ignore_index=True)
df.sort_values(by = 'Like', ascending=False).head()

NameError: name 'df' is not defined

In [24]:
scheme, f, w, d = 'f = w = d', 1/3, 1/3, 1/3
df['Like'] = f*df['F'] + w*df['W'] + d*df['D']
gdata[scheme] = df.sort_values(by = 'Like', ascending=False)['Like'].values.tolist()
gnsdata[scheme] = df['Like'].values.tolist()
for i in range(5):
    cdata = cdata.append({'Cluster':3, 
                          'Latitude': td.loc[i,'Latitude'], 'Longitude': td.loc[i,'Longitude'], 
                          'Borough': td.loc[i,'Borough'], 'Neighbourhood': td.loc[i,'Neighbourhood']},
           ignore_index=True)
df.sort_values(by = 'Like', ascending=False).head()

NameError: name 'df' is not defined

In [25]:
%matplotlib inline

gd = pd.DataFrame(data = gdata)
ax = gd.plot()
ax.set_ylabel("Level of pleasant living")
ax.set_xlabel("Different 'Borough - Neighbourhood' of Toronto")

TypeError: Empty 'DataFrame': no numeric data to plot

In [26]:

gd = pd.DataFrame(data = gnsdata)
ax = gd.plot()
ax.set_ylabel("Level of pleasant living")
ax.set_xlabel("Number 'Borough - Neighbourhood' from df-DataFrame")

TypeError: Empty 'DataFrame': no numeric data to plot

In [27]:
gd = pd.DataFrame(data = gnsdata)
for i in range(len(gd.columns)):
    gd.sort_values(by = gd.columns[i], ascending=False, inplace = True)
    gd.iloc[5:,i] = 0
    gd.sort_index(ascending=True, inplace=True)
ax = gd.plot()
ax.set_ylabel("Level of pleasant living")
ax.set_xlabel("Top-5 'Borough - Neighbourhood' from df-DataFrame")

TypeError: Empty 'DataFrame': no numeric data to plot

In [28]:
# create map
latitude, longitude = 43.650943, -79.554724
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
kclusters = 4
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cdata['Latitude'], cdata['Longitude'], cdata['Borough'] +"-"+ cdata['Neighbourhood'], cdata['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'cm' is not defined


7. Discussion section

7.1. As can be seen from graph 6.6, the largest number of places to stay, from which all family members can enjoy, is achieved in the case when the main person in the family is a wife.
The worst case is that - the main person in the family is the husband. In this case, finding a place becomes very difficult.

7.2. As can be seen from graph 6.7, the same area has a completely different level of attractiveness for life from the position of different family members.
Moreover, the requirements of the husband are met the worst in almost all areas.

7.3. As can be seen from graph 6.8, the areas best suited for living from the point of view of various family members do not practically intersect.
Of the 16 districts selected in four schemes, only 4 identical ones were found.

8. Conclusion section

Data analysis is a great tool.
It allows you to make interesting discoveries and find unexpected results.